In [38]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
! wget https://raw.githubusercontent.com/Chando0185/Emial_Spam_Classification/main/spam.csv

--2021-08-06 07:01:23--  https://raw.githubusercontent.com/Chando0185/Emial_Spam_Classification/main/spam.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503673 (492K) [text/plain]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.87K  --.-KB/s    in 0.02s   

2021-08-06 07:01:24 (28.9 MB/s) - ‘spam.csv’ saved [503673/503673]



In [5]:
df = pd.read_csv("spam.csv", encoding= 'latin-1')
df.head()

,class,message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis='columns', inplace=True)
df.head()

,class,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
sp = pd.get_dummies(df['class'],drop_first = True)
df['class'].head()

0     ham
1     ham
2    spam
3     ham
4     ham
Name: class, dtype: object

In [8]:
sp.head()

,spam
0,0
1,0
2,1
3,0
4,0


In [9]:
df.drop('class', axis = 1, inplace = True)
df['class'] = sp
df.head()

,message,class
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [10]:
df.shape

(5572, 2)

In [11]:
df.drop_duplicates(inplace = True)
df.shape

(5169, 2)

In [12]:
df.isnull().sum()

message    0
class      0
dtype: int64

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
def process(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  c_wrd = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')] 
  return c_wrd


In [21]:
df['message'].head().apply(process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [23]:
bow = CountVectorizer(analyzer=process).fit_transform(df['message'])

In [31]:
bow.shape

(5169, 11304)

In [32]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split( bow, df['class'], test_size = 0.2, random_state = 0 )

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report , accuracy_score
lr = LogisticRegression()
lr.fit ( X_train , y_train )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [48]:
predictions = lr.predict(X_test)

In [49]:
accuracy_score(y_test,predictions)

0.9777562862669246

In [50]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[885,   0],
       [ 23, 126]])

In [51]:
y_test.iloc[35]

1

In [52]:
predictions[35]

1